![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.png)
# Pipeline 4 Notebook - AutoAI Notebook v1.15.13

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimum results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/analyze-data/autoai-notebook.html">Cloud Pak for Data</a>. 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for getting data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.9 and scikit-learn 1.0.2.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data](#read)<br>
&nbsp;&nbsp;[Train and test data split](#split)<br>
&nbsp;&nbsp;[Make pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
**[Summary and next steps](#summary_and_next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs,
 - scikit-learn,
 - xgboost.


In [1]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install -U autoai-libs==1.13.2 | tail -n 1
!pip install -U scikit-learn==1.0.2 | tail -n 1
!pip install -U xgboost==1.5.1 | tail -n 1

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [2]:
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import FSLocation

training_data_references = [
    DataConnection(
        data_asset_id='698e18e9-13b6-4210-bc99-1b2a3cc20300'
    ),
]
training_result_reference = DataConnection(
    location=FSLocation(
        path='/projects/1bdf6a0d-6d94-4df4-8ef4-0f39b726584a/assets/auto_ml/82fdf6cf-628f-41b0-b703-1ab2fd5651bd/wml_data/91ee3fd2-a85e-4845-8748-31b8d43b338a/data/automl'
    )
)

Following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [3]:
experiment_metadata = dict(
    prediction_type='binary',
    prediction_column='satisfaction',
    holdout_size=0.1,
    scoring='roc_auc',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=3,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    include_only_estimators=['RandomForestClassifier', 'LogisticRegression', 'ExtraTreesClassifier', 'XGBClassifier', 'LGBMClassifier', 'GradientBoostingClassifier'],
    project_id='1bdf6a0d-6d94-4df4-8ef4-0f39b726584a',
    text_processing=False,
    train_sample_columns_index_list=[2, 4, 5, 6, 11, 12, 13, 14, 15, 16, 17, 19, 22, 23, 24],
    positive_label=1,
    drop_duplicates=True,
    retrain_on_holdout=False
)

## Set `n_jobs` parameter to the number of available CPUs

In [4]:
import os, ast
CPU_NUMBER = 2
if 'RUNTIME_HARDWARE_SPEC' in os.environ:
    CPU_NUMBER = int(ast.literal_eval(os.environ['RUNTIME_HARDWARE_SPEC'])['num_cpu'])

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

In [5]:
import os

wml_credentials = {
    "instance_id": "openshift",
    "token": os.environ.get("USER_ACCESS_TOKEN"),
    "url": os.environ.get("RUNTIME_ENV_APSX_URL"),
    "version": "4.0"
}

In [6]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(wml_credentials)

if 'space_id' in experiment_metadata:
    wml_client.set.default_space(experiment_metadata['space_id'])
else:
    wml_client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0]._wml_client = wml_client

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.  
**Note**: If reading data using Flight Server for data connections results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`. It may be necessary to use methods for initial data pre-processing like: e.g. `DataFrame.dropna()`, `DataFrame.drop_duplicates()`, `DataFrame.sample()`.

In [7]:
train_X, test_X, train_y, test_y = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True)

<a id="preview_model_to_python_code"></a>
## Make pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [8]:
from autoai_libs.transformers.exportable import ColumnSelector
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TAM
import autoai_libs.cognito.transforms.transform_extras
from autoai_libs.cognito.transforms.transform_utils import FS1
from xgboost import XGBClassifier

#### Pre-processing & Estimator.

In [9]:
column_selector_0 = ColumnSelector(
    columns_indices_list=[
        2, 4, 5, 6, 11, 12, 13, 14, 15, 16, 17, 19, 22, 23, 24,
    ]
)
numpy_column_selector_0 = NumpyColumnSelector(
    columns=[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "float_int_num", "float_int_num", "float_int_num", "float_int_num",
        "float_int_num", "float_int_num", "float_int_num", "float_int_num",
        "float_int_num", "float_int_num", "float_int_num", "float_int_num",
        "float_int_num", "float_int_num",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[[], [], [], [], [], [], [], [], [], [], [], [], [], []],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=100001
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=100001,
    filling_values_list=[
        100001, 100001, 100001, 100001, 100001, 100001, 100001, 100001,
        100001, 100001, 100001, 100001, 100001, 100001,
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    missing_values=100001,
    sklearn_version_family="1",
    strategy="most_frequent",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="1",
)
pipeline_0 = make_pipeline(
    column_selector_0,
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
column_selector_1 = ColumnSelector(
    columns_indices_list=[
        2, 4, 5, 6, 11, 12, 13, 14, 15, 16, 17, 19, 22, 23, 24,
    ]
)
numpy_column_selector_1 = NumpyColumnSelector(columns=[2])
float_str2_float = FloatStr2Float(
    dtypes_list=["float_int_num"], missing_values_reference_list=[]
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
num_imputer = NumImputer(missing_values=float("nan"), strategy="median")
opt_standard_scaler = OptStandardScaler(use_scaler_flag=False)
pipeline_1 = make_pipeline(
    column_selector_1,
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 2],
)
tam_0 = TAM(
    tans_class=autoai_libs.cognito.transforms.transform_extras.IsolationForestAnomaly,
    name="isoforestanomaly",
    col_names=[
        "customer_type", "type_of_travel", "flight_distance",
        "inflight_wifi_service", "online_boarding", "seat_comfort",
        "inflight_entertainment", "on_board_service", "leg_room_service",
        "baggage_handling", "checkin_service", "cleanliness",
        "class_business", "class_economy", "class_premium economy",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 15),
    additional_col_count_to_keep=15,
    ptype="classification",
)
tam_1 = TAM(
    tans_class=autoai_libs.cognito.transforms.transform_extras.IsolationForestAnomaly,
    name="isoforestanomaly",
    col_names=[
        "customer_type", "type_of_travel", "flight_distance",
        "inflight_wifi_service", "online_boarding", "seat_comfort",
        "inflight_entertainment", "on_board_service", "leg_room_service",
        "baggage_handling", "checkin_service", "cleanliness",
        "class_business", "class_economy", "class_premium economy",
        "isoforestanomaly_0",
    ],
    col_dtypes=[
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("float32"), np.dtype("float32"), np.dtype("float32"),
        np.dtype("int64"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 15),
    additional_col_count_to_keep=15,
    ptype="classification",
)
xgb_classifier = XGBClassifier(
    base_score=0.5,
    booster="gbtree",
    colsample_bylevel=1,
    colsample_bynode=1,
    colsample_bytree=1,
    gamma=0,
    gpu_id=-1,
    interaction_constraints="",
    learning_rate=0.06255016342132116,
    max_delta_step=0,
    max_depth=7,
    min_child_weight=18,
    missing=float("nan"),
    monotone_constraints="()",
    n_estimators=245,
    n_jobs=CPU_NUMBER,
    num_parallel_tree=1,
    predictor="auto",
    random_state=33,
    reg_alpha=0,
    reg_lambda=0.26792984834630523,
    scale_pos_weight=1,
    subsample=0.9828823385600849,
    tree_method="hist",
    validate_parameters=1,
    verbosity=0,
    nthread=2,
    silent=True,
    seed=33,
)


#### Pipeline.

In [10]:
pipeline = make_pipeline(
    union, numpy_permute_array, tam_0, fs1_0, tam_1, fs1_1, xgb_classifier
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [11]:
from sklearn.metrics import get_scorer

scorer = get_scorer(experiment_metadata['scoring'])

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [13]:
pipeline.fit(train_X.values, train_y.values);

IndexError: index 24 is out of bounds for axis 1 with size 24

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [14]:
score = scorer(pipeline, test_X.values, test_y.values)
print(score)

IndexError: index 24 is out of bounds for axis 1 with size 24

In [15]:
pipeline.predict(test_X.values)

IndexError: index 24 is out of bounds for axis 1 with size 24

<a id="summary_and_next_steps"></a>
# Summary and next steps
You successfully completed this notebook!
You learned how to use AutoAI pipeline definition to train the model.
Check out our [Online Documentation](https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/cpd/overview/relnotes-4.0.html) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2022 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___